## <i> Raw csv, wav file to pickle

In [2]:
import os
import sys
import pickle as pkl
from pathlib import Path
import numpy as np
import pandas as pd
from utils import vggish_input, params

# Configuration
PROJECT_ROOT = Path.cwd()
sys.path.append(str(PROJECT_ROOT / ".." / ".." / ".." / "HCAR"))

# Set project paths
DATA_PATH = Path("../../Data/Train_Data/1_RawDataset")
SAVE_PATH = Path("../../Data/Train_Data/2_TrainingDataset")

# Configuration
participant_list = ['3', '4', '10', '100', '101', '102', '103', '104']  # Fill with participant folder names
hand = 'Right'
class_names = ['Tooth_brushing', 'Washing_hands', 'Shower', 'Wiping', 'Vacuum_Cleaner', 'Other']
trial_counter = {}

# Process each participant's data
for participant in participant_list:
    hand_path = DATA_PATH / participant / hand
    csv_files = sorted([f for f in os.listdir(hand_path) if f.endswith("SensorData.csv")])
    audio_files = sorted([f for f in os.listdir(hand_path) if f.endswith("AudioData.csv")])

    print(f"Processing participant: {participant}, hand: {hand}")

    for csv_file, audio_file in zip(csv_files, audio_files):
        if csv_file.rsplit('_', 1)[0] != audio_file.rsplit('_', 1)[0]:
            print(f"Mismatched filenames: {csv_file} vs {audio_file}")
            continue

        activity = "_".join(csv_file.split("_")[1:-7])
        if activity not in class_names:
            print(f"[{activity}] not in predefined class list.")
            continue

        # Trial count
        key = (participant, hand, activity)
        trial_counter[key] = trial_counter.get(key, 0) + 1
        trial_num = trial_counter[key]

        # Load and preprocess IMU data
        imu_df = pd.read_csv(hand_path / csv_file)
        end_time = round(imu_df['Time'].iloc[-1])
        imu_df = imu_df[(imu_df['Time'] > 5) & (imu_df['Time'] <= end_time - 5)]

        selected_cols = ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'RotVecX', 'RotVecY', 'RotVecZ']
        imu_array = imu_df[selected_cols].to_numpy(dtype=np.float64)

        # Load and preprocess audio data
        audio_df = pd.read_csv(hand_path / audio_file, engine='python')
        audio_df = audio_df[(audio_df['Time'] > 5) & (audio_df['Time'] <= end_time - 5)]
        audio_array = audio_df.iloc[:, 8:].to_numpy(dtype=np.int64).flatten()

        # Create multimodal data dict
        multimodal_data = {
            'IMU': imu_array,
            'Audio': audio_array
        }

        # Save to pickle
        save_dir = SAVE_PATH / participant / hand
        save_dir.mkdir(parents=True, exist_ok=True)
        filename = f"{participant}---{activity}---{trial_num}.pkl"

        with open(save_dir / filename, 'wb') as f:
            pkl.dump(multimodal_data, f)

        print("Saved:", save_dir / filename)


Processing participant: 3, hand: Right
Processing participant: 4, hand: Right
Processing participant: 10, hand: Right
Processing participant: 100, hand: Right
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Other---1.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Other---2.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Other---3.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Shower---1.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Shower---2.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Tooth_brushing---1.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Vacuum_Cleaner---1.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Washing_hands---1.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Wiping---1.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDataset\100\Right\100---Wiping---2.pkl
Saved: ..\..\Data\Train_Data\2_TrainingDa

## <i> pickle 파일 전처리

In [3]:
# Generate fixed-length IMU/Audio training examples
# Sampling and framing parameters
imu_sr = 50
sub_sr = 16000
stride = 0.2
win_len_imu = 2 * imu_sr
hop_len_imu = int(imu_sr * stride)

DATA_DIR = Path("../../Data/Train_Data/2_TrainingDataset")
SAVE_DIR = Path("../../Data/Train_Data/3_MMExamples")
DATA_DIR.mkdir(parents=True, exist_ok=True)
SAVE_DIR.mkdir(parents=True, exist_ok=True)

def frame(data, win_len, hop_len):
    """Frame a time series using a sliding window with zero-padding if too short."""
    if data.shape[0] < win_len:
        pad_len = int(np.ceil(win_len)) - data.shape[0]
        data = np.concatenate([data, np.zeros((pad_len,) + data.shape[1:])], axis=0)

    n_samples = data.shape[0]
    n_frames = 1 + int(np.floor((n_samples - win_len) / hop_len))
    shape = (n_frames, win_len) + data.shape[1:]
    strides = (data.strides[0] * hop_len,) + data.strides

    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

# Main loop
chunk_minutes = 10
chunk_size = sub_sr * chunk_minutes * 60

for pid in participant_list:
    for hand_dir in (DATA_DIR / str(pid)).iterdir():
        for pkl_file in hand_dir.glob("*.pkl"):
            with open(pkl_file, "rb") as f:
                raw_data = pkl.load(f)

            print(f"Processing {pkl_file.name} for participant {pid}")

            imu_windows = frame(raw_data["IMU"], win_len_imu, hop_len_imu)
            wav_raw = raw_data["Audio"][::16000 // sub_sr].astype(np.int16)

            save_path = SAVE_DIR / str(pid) / hand_dir.name / str(sub_sr)
            save_path.mkdir(parents=True, exist_ok=True)

            audio_chunks = [
                wav_raw[i:i + chunk_size]
                for i in range(0, len(wav_raw), chunk_size)
            ]

            audio_features = [
                vggish_input.wavform_to_concat_examples(chunk, 10, sub_sr // 2, sr=sub_sr)
                for chunk in audio_chunks
            ]
            audio_features = np.concatenate(audio_features, axis=0)

            # Align audio features with IMU windows
            audio_timestamps = [
                i * params.STFT_HOP_LENGTH_SECONDS + params.STFT_WINDOW_LENGTH_SECONDS
                for i in range(audio_features.shape[0])
            ]

            audio_data, imu_data = [], []
            for i in range(imu_windows.shape[0]):
                end_imu = i * hop_len_imu + win_len_imu
                end_time = end_imu / imu_sr

                j = int((end_time - params.STFT_WINDOW_LENGTH_SECONDS) / params.STFT_HOP_LENGTH_SECONDS)
                sample_window = int(params.EXAMPLE_WINDOW_SECONDS / params.STFT_HOP_LENGTH_SECONDS)
                start = j - sample_window

                if start < 0:
                    continue

                audio_sample = audio_features[start:j]
                if audio_sample.shape[0] < sample_window:
                    pad = np.zeros((sample_window - audio_sample.shape[0],) + audio_sample.shape[1:])
                    audio_sample = np.concatenate([audio_sample, pad], axis=0)

                audio_data.append(audio_sample)
                imu_data.append(imu_windows[i])

            imu_arr = np.array(imu_data)
            audio_arr = np.array(audio_data)

            assert imu_arr.shape[0] == audio_arr.shape[0], f"Shape mismatch in {pkl_file.name}"

            dataset = {
                "IMU": imu_arr,
                "audio": audio_arr
            }

            with open(save_path / pkl_file.name, "wb") as f:
                pkl.dump(dataset, f)

Processing 3---Other---1.pkl for participant 3
Processing 3---Shower---1.pkl for participant 3
Processing 3---Tooth_brushing---1.pkl for participant 3
Processing 3---Tooth_brushing---10.pkl for participant 3
Processing 3---Tooth_brushing---2.pkl for participant 3
Processing 3---Tooth_brushing---3.pkl for participant 3
Processing 3---Tooth_brushing---4.pkl for participant 3
Processing 3---Tooth_brushing---5.pkl for participant 3
Processing 3---Tooth_brushing---6.pkl for participant 3
Processing 3---Tooth_brushing---7.pkl for participant 3
Processing 3---Tooth_brushing---8.pkl for participant 3
Processing 3---Tooth_brushing---9.pkl for participant 3
Processing 3---Vacuum_Cleaner---1.pkl for participant 3
Processing 3---Washing_hands---1.pkl for participant 3
Processing 3---Washing_hands---10.pkl for participant 3
Processing 3---Washing_hands---2.pkl for participant 3
Processing 3---Washing_hands---3.pkl for participant 3
Processing 3---Washing_hands---4.pkl for participant 3
Processing 3